# Testing S-57 Data Processing Classes

## 1. Setup

In [2]:
import sys
import os
from distutils.command.config import config
from pathlib import Path
from dotenv import load_dotenv
# Add the src directory to the Python path
project_root = Path.cwd().parent.parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Load environment variables from .env file at the project root
load_dotenv(project_root / ".env")

from src.maritime_module.core.s57_data import S57Base, S57Advanced, S57Updater, PostGISManager, SpatiaLiteManager, GPKGManager, S57AdvancedConfig
# Define paths for data and output
s57_data_dir = project_root / 'data' / 'ENC_ROOT'
s57_data_update_dir = project_root / 'data' / 'ENC_ROOT_UPDATE'
output_dir = Path.cwd() / 'output'
output_dir.mkdir(exist_ok=True)

# Define database parameters from environment variables
db_params = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}

print(f"S-57 data directory: {s57_data_dir}")
print(f"Output directory: {output_dir}")
print(f"PostGIS DB_name: {db_params['dbname']}")

/tmp/ipykernel_3379/1886726326.py:3: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.command.config import config


S-57 data directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/data/ENC_ROOT
Output directory: /home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/docs/notebooks/output
PostGIS DB_name: ENC_db


In [ ]:
from src.maritime_module.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params)
pg.connect()
print(pg.get_schemas())

In [ ]:
from osgeo import gdal
print("GDAL Python bindings version:", gdal.__version__)
print("GDAL C library version:", gdal.VersionInfo("RELEASE_NAME"))

## 2. Test S57Base: Bulk Conversion

#### Convert to GPKG

In [ ]:
# import logging
# logging.getLogger('src.maritime_module.core.s57_data').setLevel(logging.DEBUG)

base_converter = S57Base(
        input_path=s57_data_dir,
        output_dest=str(output_dir / 'by_enc'),
        output_format='gpkg',
        overwrite=True
    )
base_converter.convert_by_enc()

#### Convert to PostGIS

In [ ]:
base_converter = S57Base(
        input_path=s57_data_dir,
        output_dest=db_params,
        output_format='postgis',
        overwrite=False
    )
base_converter.convert_by_enc()

In [ ]:
from src.maritime_module.utils.db_utils import PostGISConnector

pg = PostGISConnector(db_params)
pg.connect()
pg.get_schema_summary()

#### Convert to SpatiaLite

In [ ]:
base_converter = S57Base(
        input_path=s57_data_dir,
         output_dest=str(output_dir / 'by_enc'),
        output_format='spatialite',
        overwrite=True
    )
base_converter.convert_by_enc()

## 3. Test S57Advanced: Layer-centric Conversion

This section provides comprehensive tests for the `S57Advanced` class, which performs optimized, layer-centric conversions. We will test conversions to all supported formats (PostGIS, GeoPackage, SpatiaLite) and verify the results.

### 3.1. Convert to PostGIS and Verify

First, we convert the S-57 data into a layer-centric PostGIS database schema. Then, we use `PostGISManager` to verify the output.

In [3]:
# --- Step 1: Conversion ---
pg_schema = 's57_advanced'
print(f"--- Starting S57Advanced conversion to PostGIS schema: '{pg_schema}' ---")

advanced_converter_pg = S57Advanced(
    input_path=s57_data_dir,
    output_dest=db_params,
    output_format='postgis',
    overwrite=True,
    schema=pg_schema,
    config=S57AdvancedConfig(auto_tune_batch_size=True, enable_debug_logging=False)
)
advanced_converter_pg.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = PostGISManager(db_params=db_params, schema=pg_schema)

    # Verify a common layer exists and has data
    depare_layer = manager.get_layer('lndmrk')
    print(f"Loaded {len(depare_layer)} features from LNDMRK layer.")
    print("LNDMRK layer head (note the 'dsid_*' stamping columns):")
    display(depare_layer.head())

    # Verify feature stamping integrity across all layers
    print("\nVerifying feature update status against DSID table...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during PostGIS verification: {e}")


--- Starting S57Advanced conversion to PostGIS schema: 's57_advanced' ---
2025-09-15 21:37:19,587 - src.maritime_module.core.s57_data - INFO - Found 6 S-57 file(s).
2025-09-15 21:37:19,588 - src.maritime_module.core.s57_data - INFO - --- Starting optimized 'by_layer' conversion ---
2025-09-15 21:37:19,589 - src.maritime_module.core.s57_data - INFO - Files to process: 6, Batch size: 3
2025-09-15 21:37:19,589 - src.maritime_module.core.s57_data - INFO - Pre-processing files to extract schemas and ENC names...
2025-09-15 21:37:19,706 - src.maritime_module.core.s57_data - INFO - Built unified schemas for 86 layers
2025-09-15 21:37:19,804 - src.maritime_module.utils.db_utils - INFO - Successfully connected to database 'ENC_db' for schema management.
2025-09-15 21:37:19,817 - src.maritime_module.utils.db_utils - WARNING - Overwrite is enabled. Dropping and recreating schema 's57_advanced'...
2025-09-15 21:37:19,925 - src.maritime_module.utils.db_utils - INFO - Schema 's57_advanced' is ready.

,ogc_fid,rcid,prim,grup,objl,rver,agen,fidn,fids,lnam,...,scamin,txtdsc,recdat,recind,sordat,sorind,dsid_dsnm,dsid_edtn,dsid_updn,wkb_geometry
0,1,283.0,1.0,2.0,74.0,1.0,550.0,22330302.0,50.0,02260154BBBE0032,...,499999,None,None,None,20090519,"US,US,reprt,11thCGD,LNM 20/09",US3CA52M,31,0,POINT (-121.90155 36.30625)
1,2,284.0,1.0,2.0,74.0,1.0,550.0,18686117.0,50.0,0226011D20A50032,...,499999,None,None,None,20120402,"US,US,graph,GC-11734",US3CA52M,31,0,POINT (-121.78198 36.80455)
2,3,285.0,1.0,2.0,74.0,1.0,550.0,18686123.0,50.0,0226011D20AB0032,...,499999,None,None,None,201305,"US,US,graph,Chart 18680",US3CA52M,31,0,POINT (-121.90531 36.96865)
3,4,286.0,1.0,2.0,74.0,1.0,550.0,386139899.0,12345.0,0226170406FB3039,...,499999,None,None,None,20150114,"US,US,reprt,L-95/15",US3CA52M,31,0,POINT (-121.98162 36.96178)
4,5,287.0,1.0,2.0,74.0,1.0,550.0,18686120.0,50.0,0226011D20A80032,...,499999,None,None,None,201305,"US,US,graph,Chart 18680",US3CA52M,31,0,POINT (-122.03182 36.97857)



Verifying feature update status against DSID table...
2025-09-15 21:37:38,422 - src.maritime_module.core.s57_data - INFO - Feature update status verification complete: {'VALID': np.int64(246)}
Verification Summary:


,layer_name,enc_name,dsid_edition,dsid_update,feature_edition,feature_update,feature_count,edition_match,update_match,status
216,achare,US1GC09M,65,5,65,5,1,True,True,VALID
217,achare,US4CA60M,37,6,37,6,3,True,True,VALID
1,admare,US1EEZ1M,9,0,9,0,11,True,True,VALID
0,admare,US1GC09M,65,5,65,5,6,True,True,VALID
3,admare,US1PO02M,21,1,21,1,1,True,True,VALID
...,...,...,...,...,...,...,...,...,...,...
180,wedklp,US3CA52M,31,0,31,0,61,True,True,VALID
181,wedklp,US4CA60M,37,6,37,6,122,True,True,VALID
184,wrecks,US1GC09M,65,5,65,5,148,True,True,VALID
182,wrecks,US3CA52M,31,0,31,0,9,True,True,VALID


### 3.2. Convert to GeoPackage and Verify

Next, we test the conversion to a single GeoPackage file. The `GPKGManager` is used for verification.

In [ ]:
# --- Step 1: Conversion ---
gpkg_path = output_dir / 's57_advanced.gpkg'
print(f"--- Starting S57Advanced conversion to GeoPackage: '{gpkg_path}' ---")

advanced_converter_gpkg = S57Advanced(
    input_path=s57_data_dir,
    output_dest=str(gpkg_path),
    output_format='gpkg',
    overwrite=True,
    config=S57AdvancedConfig(auto_tune_batch_size=True)
)
advanced_converter_gpkg.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = GPKGManager(gpkg_path=gpkg_path)

    # Verify a common layer exists and has data
    soundg_layer = manager.get_layer('lndmrk')
    print(f"Loaded {len(soundg_layer)} features from LNDMRK layer in the GeoPackage.")
    print("LNDMRK layer head:")
    display(soundg_layer.head())

    # Verify feature stamping integrity
    print("\nVerifying feature update status...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during GeoPackage verification: {e}")


### 3.3. Convert to SpatiaLite and Verify

Finally, we test the conversion to a SpatiaLite database file and verify it with `SpatiaLiteManager`.

In [ ]:
# --- Step 1: Conversion ---
sqlite_path = output_dir / 's57_advanced.sqlite'
print(f"--- Starting S57Advanced conversion to SpatiaLite: '{sqlite_path}' ---")

advanced_converter_sqlite = S57Advanced(
    input_path=s57_data_dir,
    output_dest=str(sqlite_path),
    output_format='spatialite',
    overwrite=True,
    config=S57AdvancedConfig(auto_tune_batch_size=True)
)
advanced_converter_sqlite.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = SpatiaLiteManager(db_path=sqlite_path)

    # Verify a common layer exists and has data
    boyspp_layer = manager.get_layer('boyspp')
    print(f"Loaded {len(boyspp_layer)} features from BOYSPP layer in the SpatiaLite DB.")
    print("BOYSPP layer head:")
    display(boyspp_layer.head())

    # Verify feature stamping integrity
    print("\nVerifying feature update status...")
    verification_results = manager.verify_feature_update_status()
    print("Verification Summary:")
    display(verification_results)

except Exception as e:
    print(f"An error occurred during SpatiaLite verification: {e}")


### 3.4. Test with Enterprise-Safe Parallel Processing

This test demonstrates using the `S57AdvancedConfig` to enable high-safety, read-only parallel processing, which can speed up the initial file discovery and preprocessing steps.

In [ ]:
# --- Step 1: Configure and Convert ---
parallel_schema = 's57_parallel_test'
print(f"--- Starting S57Advanced conversion with PARALLEL processing to schema: '{parallel_schema}' ---")

# Use the recommended high-safety configuration for parallel reads
high_safety_config = S57AdvancedConfig(
    enable_parallel_processing=True,
    parallel_read_only=True,
    parallel_db_writes=False,
    parallel_validation_level='strict',
    max_parallel_workers=2,
    enable_debug_logging=True
)

print("\nUsing configuration:")
print(high_safety_config.get_configuration_summary())

advanced_converter_parallel = S57Advanced(
    input_path=s57_data_dir,
    output_dest=db_params,
    output_format='postgis',
    overwrite=True,
    schema=parallel_schema,
    config=high_safety_config
)
advanced_converter_parallel.convert_to_layers()

print("\n--- Step 2: Verification ---")
# --- Step 2: Verification ---
try:
    manager = PostGISManager(db_params=db_params, schema=parallel_schema)
    summary = manager.get_enc_summary()
    print(f"Successfully created {len(summary)} ENCs in the parallel-processed schema.")
    display(summary.head())
except Exception as e:
    print(f"An error occurred during parallel processing verification: {e}")


## 4. Test S57Updater

The `S57Updater` is designed to perform incremental, transactional updates on a layer-centric database created by `S57Advanced`. We will use the schema created in test 3.1 (`s57_advanced_test`) as the target for our update.

**Note:** This test requires a running PostGIS database.

Update check:
Old Edition  -> New Edition |
US1EEZ1M 9:0  -> 9:0 |
US1GC09M 65:5 -> 71:1 |
US1PO02M 21:1 -> 21:1 |
US2WC12M 27:8 -> 27:12 |
US3CA52M 31:0 -> 31:7 |
US4CA60M 37:6 -> 38:2

### PostGIS Updater

In [ ]:
db_schema = 's57_advanced' # Use the schema created by S57Advanced
# Example S-57 file to update,
updater = S57Updater(output_format='postgis',
                     dest_conn=db_params,
                     schema=db_schema,
                     )
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()
display(summary_df)

### 4.2. SpatiaLite Updater

This test demonstrates the `S57Updater` functionality for a SpatiaLite database. The previous implementation failed with a `database disk image is malformed` error because it attempted to run the updater on a file that was being accessed by multiple, uncoordinated connections (OGR and SQLAlchemy).

The corrected test follows a robust, isolated process:

1.  **Create a fresh SpatiaLite database** from the initial ENC data (`s57_data_dir`). This provides a clean, known state to update and prevents file corruption.
2.  **Run the updater** on this new database using the update data (`s57_data_update_dir`).
3.  **Verify the results** by comparing the ENC versions before and after the update and checking for data integrity.

In [ ]:
sqlite_update_target_path = output_dir / 's57_advanced.sqlite'
# Example S-57 file to update,
updater = S57Updater(
     output_format='spatialite',
     dest_conn=str(sqlite_update_target_path),
     schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
)
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()

### GPKG Updater

In [ ]:
update_target_path = output_dir / 's57_advanced.gpkg'
# Example S-57 file to update,
updater = S57Updater(
     output_format='gpkg',
     dest_conn=str(update_target_path),
     schema='main' # SpatiaLite doesn't use schemas like PostGIS, but 'main' is the default
)
#update_new = updater.update_from_location(s57_data_update_dir)
update_force = updater.force_update_from_location(s57_data_dir, enc_filter=['US3CA52M', 'US1GC09M'] ) # Force reinstall specific problematic ENCs
summary_df = updater.get_change_summary()

## DeepTest

#### Setup

In [ ]:
from tests.core__real_data.deep_test_s57_workflow import *

db_params_test = {
    'dbname': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db_schema = 's57_deeptest'
test_output_path = project_root / 'test_output'

test_config = TestConfig(s57_data_root = s57_data_dir,
                         s57_update_root=s57_data_update_dir,
                         test_output_dir = test_output_path,
                         test_level = 3,

                         skip_postgis = False,
                         skip_updates = False,
                         cleanup_on_success = True,
                         postgis_config = db_params_test,
                         test_schema_name = db_schema,
                         clean_output = True,
                         exclude_extra_cols= ["geometry","geom", "wkb_geometry"],
)


print(f"\n--- Starting S57 Deep Test process with TEST CONFIG ---")
print(f"\n--- SettingUp directories:")
print(f"\nTest dataset at:       {test_config.s57_data_root}")
print(f"\nUpdate dataset at:     {test_config.s57_update_root}")
print(f"\nTest output directory: {test_config.test_output_dir}")
print(f"\n-----------------------------------")
post_conf = (f"Dababase: {db_params_test['dbname']} | Schema/Filename: {db_schema}") if  test_config.skip_postgis == False else ""
print(f"\nPostGIS tests:   {'❌' if test_config.skip_postgis == True else '✅'} {post_conf}")
print(f"\nUpdates process: {'❌' if test_config.skip_updates == True else '✅'}")


#### Load Dataset

In [2]:
try:
    print(f"\n🎉 Commence DeepTest !")
    tester = S57DeepTester(test_config)
except Exception as e:
    logger.error(f"DeepTest execution failed: {e}", exc_info=True)
    sys.exit(1)

# Testing Update Readiness
if test_config.skip_updates == False:
    compare_df = tester.analyze_update_readiness()
    display(compare_df)



🎉 Commence DeepTest !


NameError: name 'logger' is not defined

In [ ]:
try:
    report = tester.run_comprehensive_test()

    print(f"\n🎉 DeepTest completed successfully!")
    print(f"📁 Results saved to: {test_config.test_output_dir}")

except Exception as e:
    logger.error(f"DeepTest execution failed: {e}", exc_info=True)
    sys.exit(1)

In [2]:
import geopandas as gpd
import fiona

test_output_path = project_root / "tests" / "core__real_data" /"test_output"
# test_file = test_output_path / "s57_deeptest.gpkg"
test_file = test_output_path / "s57_deeptest.sqlite"
print(test_output_path)
print(test_file)
# --- 1. List all layers in the GeoPackage file ---
try:
    layer_names = fiona.listlayers(test_file)
    print(f"Layers found in '{test_file.name}':")
    for name in layer_names:
        print(f"- {name}")
except fiona.errors.DriverError as e:
    print(f"Error: Could not open the file '{test_file}'. Please check the path and ensure it's a valid GeoPackage file.")
    print(f"Details: {e}")
    # Exit if we can't open the file
    exit()

# --- 2. Select and read a specific layer ---
# Replace 'your_layer_name' with the name of the layer you want to open
# For example, if you have a layer named 'buildings', you would use:
# selected_layer_name = 'buildings'
selected_layer_name = 'lndmrk'

if selected_layer_name in layer_names:
    print(f"\nReading layer: '{selected_layer_name}'...")

    # Use geopandas.read_file with the layer parameter
    gdf = gpd.read_file(test_file, layer=selected_layer_name, engine="fiona")

    # --- 3. Work with your data ---
    print(f"Successfully loaded {len(gdf)} features from '{selected_layer_name}'.")

    # Print the first 5 rows of the GeoDataFrame
    print("\nFirst 5 rows of the layer:")
    test_df = gdf[gdf['colpat'].notna()]

    # Print information about the columns and data types
    print("\nLayer information:")
    gdf.info()

else:
    print(f"\nError: Layer '{selected_layer_name}' not found in the GeoPackage file.")
    print("Please choose one of the available layers listed above.")

/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/tests/core__real_data/test_output
/home/vikont_tux/python_projects_wsl2/1_MaritimeModule_V1/tests/core__real_data/test_output/s57_deeptest.sqlite
Layers found in 's57_deeptest.sqlite':
- isolatednode
- connectednode
- edge
- face
- admare
- airare
- bcnlat
- bcnspp
- bridge
- buisgl
- buaare
- boylat
- boysaw
- boyspp
- cblsub
- ctnare
- cgusta
- coalne
- conzne
- daymar
- depare
- depcnt
- dmpgrd
- exezne
- fogsig
- hrbfac
- lakare
- lndare
- lndelv
- lndrgn
- lndmrk
- lights
- magvar
- mipare
- obstrn
- pipsol
- rdocal
- rdosta
- rectrc
- resare
- rivers
- seaare
- sbdare
- slcons
- siltnk
- slogrd
- soundg
- tesare
- topmar
- tssbnd
- tsslpt
- tsezne
- uwtroc
- wattur
- wedklp
- wrecks
- m_covr
- m_npub
- m_nsys
- m_qual
- fshzne
- achare
- cblare
- cblohd
- damcon
- fairwy
- feryrt
- marcul
- morfac
- navlne
- ofsplf
- pilpnt
- pilbop
- pipare
- ponton
- prcare
- rtpbcn
- splare
- slotop
- unsare
- canals
- ctsare
- curent



In [3]:
test_df

,rcid,prim,grup,objl,rver,agen,fidn,fids,lnam,lnam_refs,...,scamin,txtdsc,recdat,recind,sordat,sorind,dsid_dsnm,dsid_edtn,dsid_updn,geometry
79,1948,1,2,74,1,550,28063069,50,022601AC355D0032,"[022601AC355F0032, 022601AC355E0032, 022601AC3...",...,9999999,None,None,None,20111201,"US,US,graph,Chart 11524",US1GC09M,65,5,POINT (-79.84328 32.75795)
81,1950,1,2,74,1,550,450842981,6465,02261ADF51651941,"[02261E441DC01941, 022626A0CF061941]",...,9999999,None,None,None,20151104,"US,US,reprt,L-1615/2015, NGA light list #325.00",US1GC09M,65,5,POINT (-78.88337 22.68496)
91,1960,1,2,74,1,550,35445370,50,0226021CDA7A0032,[0226021CC8020032],...,9999999,None,None,None,201202,"US,US,graph,Chart 11013",US1GC09M,65,5,POINT (-83.2159 22.98921)
104,1973,1,2,74,1,550,35439108,50,0226021CC2040032,[0226021CC91C0032],...,9999999,None,None,None,201202,"US,US,graph,Chart 11013",US1GC09M,65,5,POINT (-84.02367 22.71648)
112,1981,1,2,74,1,550,35440243,50,0226021CC6730032,[0226021CD0630032],...,9999999,None,None,None,201202,"US,US,graph,Chart 11013",US1GC09M,65,5,POINT (-81.0399 21.66121)
115,1984,1,2,74,1,550,253936398,4044,02260F22C30E0FCC,"[02260F22BF4F0FCC, 02260F22BFD90FCC]",...,9999999,None,None,None,200303,"US,US,graph,Chart 411",US1GC09M,65,5,POINT (-89.69113 22.38822)
142,2011,1,2,74,1,550,8745907,50,0226008573B30032,[02260085734D0032],...,9999999,None,None,None,20030121,"US,US,reprt,7thCGD,LNM 03/03",US1GC09M,65,5,POINT (-80.54345 28.46035)
144,2013,1,2,74,1,550,253936611,4044,02260F22C3E30FCC,[02260F22C01C0FCC],...,9999999,None,None,None,200303,"US,US,graph,Chart 411",US1GC09M,65,5,POINT (-76.19634 17.91273)
145,2014,1,2,74,1,550,-1145870482,12345,0226BBB3676E3039,"[0226000006FD0001, 02263377A9D31941]",...,9999999,None,None,None,2015,"US,US,reprt,NGA light List #12676",US1GC09M,65,5,POINT (-80.48141 23.20533)
151,2020,1,2,74,1,550,368797181,6465,022615FB65FD1941,[022620822C7A1941],...,9999999,None,None,None,20151104,"US,US,reprt,L-1615/2015, NGA light list #11800",US1GC09M,65,5,POINT (-76.96487 26.53864)
